In [7]:
import pandas as pd
import requests
import json
import time
import urllib.parse

In [8]:
def json_request(url):
    response = requests.get(url)
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return response.status_code

In [9]:
cdw = pd.read_csv('https://raw.githubusercontent.com/wikimedia-research/canonical-data/master/wiki/wikis.tsv', sep='\t')
cdw['domain_name_srt'] = cdw.domain_name.str.replace('.org', '')
cdw.head(3)

/tmp/ipykernel_5123/3506203142.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  cdw['domain_name_srt'] = cdw.domain_name.str.replace('.org', '')


,database_code,domain_name,database_group,language_code,language_name,status,visibility,editability,english_name,domain_name_srt
0,aawiki,aa.wikipedia.org,wikipedia,aa,Afar,closed,public,public,Afar Wikipedia,aa.wikipedia
1,aawikibooks,aa.wikibooks.org,wikibooks,aa,Afar,closed,public,public,Afar Wikibooks,aa.wikibooks
2,aawiktionary,aa.wiktionary.org,wiktionary,aa,Afar,closed,public,public,Afar Wiktionary,aa.wiktionary


In [20]:
db_domain_map = {}
for m in cdw[['database_code', 'domain_name_srt']].to_dict('records'):
    db_domain_map[m['database_code']] = m['domain_name_srt']
    
db_name_map = {}
for m in cdw[['database_code', 'english_name']].to_dict('records'):
    db_name_map[m['database_code']] = m['english_name']

In [21]:
def key_replace(p_dict, r_dict):
    t_dict = {}
    for k in p_dict.keys():
        t_dict[r_dict[k]] = p_dict[k]
    return t_dict

In [22]:
def user_data_retreiver(username):
    
    user_data = {}
    user_data[username] = {}
    username_parsed = urllib.parse.quote(username)
    
    edits_url = f'https://meta.wikimedia.org/w/api.php?action=query&format=json&list=&meta=globaluserinfo&formatversion=2&guiuser={username_parsed}&guiprop=editcount%7Cgroups%7Cmerged%7Crights'
    json_response = json_request(edits_url)
    
    user_data[username]['home_wiki'] = db_name_map[json_response['query']['globaluserinfo']['home']]
    user_data[username]['global_edits'] = json_response['query']['globaluserinfo']['editcount']
    if len(json_response['query']['globaluserinfo']['groups']) == 0:
        user_data[username]['global_groups'] = str(None)
    else:
        user_data[username]['global_groups'] = json_response['query']['globaluserinfo']['groups']

    edits_by_wiki, groups_by_wiki, blocks_by_wiki = {}, {}, {}
    
    if json_response['query']['globaluserinfo']['editcount'] > 0:
        
        merged_wikis = json_response['query']['globaluserinfo']['merged']

        for w in merged_wikis:
            if w['editcount'] > 0:
                
                if w['wiki'] == 'wikidatawiki':
                    edits_by_wiki[w['wiki']] = w['editcount'] / 5
                else:
                    edits_by_wiki[w['wiki']] = w['editcount']
                
                try:
                    groups_by_wiki[w['wiki']] = w['groups']
                except:
                    pass
                
                try:
                    blocks_by_wiki[w['wiki']] = [w['blocked']['expiry'], w['blocked']['reason']]
                except:
                    pass                 

        top_five_wikis = dict(sorted(edits_by_wiki.items(), key=lambda x:x[1], reverse=True)[:5])
        
        me_wiki = list(top_five_wikis.keys())[0]
        user_data[username]['me_wiki'] = me_wiki
        user_data[username]['me_wiki_group'] = cdw.query("""database_code == @me_wiki""").database_group.values[0]
        user_data[username]['me_wiki_lang'] = cdw.query("""database_code == @me_wiki""").language_name.values[0]
        
        user_data[username]['edits'] = key_replace(top_five_wikis, db_name_map)
        user_data[username]['groups'] = key_replace(groups_by_wiki, db_name_map)
        
        if len(blocks_by_wiki) == 0:
            user_data[username]['blocks'] = str(None)
        else:
            user_data[username]['blocks'] = key_replace(blocks_by_wiki, db_name_map)
            
        articles_by_wiki = {}
        
        try:
            for w in top_five_wikis.keys():

                domain_srt = db_domain_map[w]
                db_group = cdw.query("""database_code == @w""").database_group.values[0]

                if db_group == 'commonswiki':
                    ns_group = [6]
                elif db_group == 'wikisource':
                    ns_group = [0, 102, 104, 106, 114]
                else:
                    ns_group = [0]
                
                n_articles_count = 0
                for ns_id in ns_group:
                    n_articles_url = f'https://xtools.wmflabs.org/api/user/pages_count/{domain_srt}/{username_parsed}/{ns_id}/noredirects/live'
                    json_response = json_request(n_articles_url)
                    try:
                        n_articles_count += json_response['counts']['count']
                    except:
                        pass
                    
                articles_by_wiki[w] = n_articles_count
            
            articles_by_wiki = dict(sorted(articles_by_wiki.items(), key=lambda x:x[1], reverse=True))
            user_data[username]['articles'] = key_replace(articles_by_wiki, db_name_map)
        
        except:
            user_data[username]['articles'] = {'failed to retreive/none'}
    
    return user_data

In [23]:
test_usernames = ['KCVelaga', 'Nivas10798', 'Satdeep Gill']
usernames = pd.DataFrame(test_usernames, columns=['username'])

In [24]:
all_user_data = {}
failed_users = []
for username in usernames.username.values:
    try:
        result = user_data_retreiver(username)
        all_user_data.update(result)
    except:
        failed_users.append(username)
print(f'Retrieval failed for {len(failed_users)} usernames.')

Retrieval failed for 0 usernames.


In [25]:
all_user_data_df = pd.DataFrame(all_user_data).transpose().reset_index().rename({'index': 'user_name'}, axis=1)
all_user_data_df.head()

,user_name,home_wiki,global_edits,global_groups,me_wiki,me_wiki_group,me_wiki_lang,edits,groups,blocks,articles
0,KCVelaga,English Wikipedia,106837,[vrt-permissions],enwiki,wikipedia,English,"{'English Wikipedia': 22964, 'Wikimedia Common...","{'Wikimedia Commons': ['filemover', 'image-rev...",None,"{'Wikidata': 4222, 'Meta-Wiki': 117, 'English ..."
1,Nivas10798,English Wikipedia,7385,None,wikidatawiki,wikidata,English,"{'Wikidata': 1239.2, 'English Wikipedia': 568,...",{'English Wikipedia': ['extendedconfirmed']},None,"{'Meta-Wiki': 18, 'Telugu Wikipedia': 7, 'Engl..."
2,Satdeep Gill,English Wikipedia,53558,[vrt-permissions],pawiki,wikipedia,Punjabi,"{'Punjabi Wikipedia': 19289, 'Wikimedia Common...","{'Wikimedia Commons': ['filemover', 'ipblock-e...",None,"{'Punjabi Wikipedia': 1398, 'Meta-Wiki': 146, ..."
